<a href="https://colab.research.google.com/github/sAndreotti/MedicalMeadow/blob/main/ATML_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [ ]:
!pip install datasets accelerate peft bitsandbytes transformers trl==0.12.0 plotly huggingface_hub
!pip install --upgrade smart_open
!pip install --upgrade gensim
!pip install ffmpeg-python
!pip install -U openai-whisper
!pip install scipy librosa unidecode inflect

In [ ]:
from datasets import load_dataset
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from trl import SFTTrainer
import re
from gensim.models.word2vec import Word2Vec
import plotly.express as px
import random
from sklearn.manifold import TSNE
from torch.utils.data import Dataset
from torch.utils.data import random_split
from peft import prepare_model_for_kbit_training, LoraConfig
from huggingface_hub import login
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from datasets import Dataset as HFDataset
from peft import AutoPeftModelForCausalLM
import whisper
from IPython.display import Audio

Import Libraries for audio part

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import scipy

## Investigate Dataset

In [ ]:
ds = load_dataset("medalpaca/medical_meadow_medical_flashcards")
ds = ds['train']
ds

In [ ]:
print(ds.features, "\n")
print("Instruction:")
print(f"length: {len(ds['instruction'])}")
print(f"example: {ds['instruction'][0]} \n")

print(f"Input:")
print(f"length: {len(ds['input'])}")
print(f"example: {ds['input'][0]} \n")

print(f"Output:")
print(f"length: {len(ds['output'])}")
print(f"example: {ds['output'][0]} \n")

### Some plots about the dataset

In [ ]:
instructions = ds['instruction']
input_phrases = ds['input']
output_phrases = ds['output']

In [ ]:
# Count the frequency of each unique instruction
instruction_counts = {instruction: instructions.count(instruction) for instruction in set(instructions)}

# Sort the instructions by frequency
sorted_instructions = sorted(instruction_counts.items(), key=lambda x: x[1], reverse=True)

# Separate the instructions and their counts for plotting
sorted_instruction_names = [item[0] for item in sorted_instructions]
sorted_instruction_counts = [item[1] for item in sorted_instructions]

# Plotting the frequency of instructions
plt.figure(figsize=(10, 5))

bars = plt.barh(sorted_instruction_names, sorted_instruction_counts, color='skyblue', edgecolor='black', linewidth=1.2)
plt.title('Instruction Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Instruction')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the length of each phrase
input_lengths = [len(phrase) for phrase in input_phrases]
output_lengths = [len(phrase) for phrase in output_phrases]

# Define the bins for the length ranges
max_input = max(input_lengths)
max_output = max(output_lengths)

input_bins = [i * max_input / 10 for i in range(1, 11)]
output_bins = [i * max_output / 10 for i in range(1, 11)]
bin_labels_input = [f'{int(input_bins[i-1])}-{int(input_bins[i])}' for i in range(1, 10)]
bin_labels_output = [f'{int(output_bins[i-1])}-{int(output_bins[i])}' for i in range(1, 10)]

# Bin the lengths into the categories
input_binned = np.digitize(input_lengths, input_bins)  # Categorize based on input lengths
output_binned = np.digitize(output_lengths, output_bins)  # Categorize based on output lengths

# Count how many phrases fall into each bin
input_bin_counts = [sum(input_binned == i) for i in range(1, len(input_bins))]
output_bin_counts = [sum(output_binned == i) for i in range(1, len(output_bins))]

# Plotting the bar charts
plt.figure(figsize=(20, 10))

# Plotting the input phrase lengths
plt.subplot(1, 2, 1)
plt.bar(bin_labels_input, input_bin_counts, color='skyblue', edgecolor='black')
plt.title('Input Phrases Length Distribution')
plt.xlabel('Length Range')
plt.ylabel('Number of Phrases')

# Plotting the output phrase lengths
plt.subplot(1, 2, 2)
plt.bar(bin_labels_output, output_bin_counts, color='skyblue', edgecolor='black')
plt.title('Output Phrases Length Distribution')
plt.xlabel('Length Range')
plt.ylabel('Number of Phrases')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate the length of each phrase
input_lengths = [len(phrase) for phrase in input_phrases]
output_lengths = [len(phrase) for phrase in output_phrases]

# Define the bins for the length ranges
max_input = max(input_lengths)
max_output = max(output_lengths)

input_bins = [i * max_input / 10 for i in range(1, 11)]
output_bins = [i * max_output / 10 for i in range(1, 11)]
bin_labels_input = [f'{int(input_bins[i-1])}-{int(input_bins[i])}' for i in range(1, 10)]
bin_labels_output = [f'{int(output_bins[i-1])}-{int(output_bins[i])}' for i in range(1, 10)]

# Bin the lengths into the categories
input_binned = np.digitize(input_lengths, input_bins)  # Categorize based on input lengths
output_binned = np.digitize(output_lengths, output_bins)  # Categorize based on output lengths

# Count how many phrases fall into each bin
input_bin_counts = [sum(input_binned == i) for i in range(1, len(input_bins))]
output_bin_counts = [sum(output_binned == i) for i in range(1, len(output_bins))]

# Plotting the bar charts with improved styling
plt.figure(figsize=(20, 10))

# Plotting the input phrase lengths
plt.subplot(1, 2, 1)
bars_input = plt.bar(bin_labels_input, input_bin_counts, color='#FF6347', edgecolor='black', linewidth=1.2)
plt.title('Input Phrases Length Distribution', fontsize=16, weight='bold', color='#2F4F4F')
plt.xlabel('Length Range', fontsize=14, color='#2F4F4F')
plt.ylabel('Number of Phrases', fontsize=14, color='#2F4F4F')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add data labels on bars for input
for bar in bars_input:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, int(yval), ha='center', fontsize=12, color='black')

# Plotting the output phrase lengths
plt.subplot(1, 2, 2)
bars_output = plt.bar(bin_labels_output, output_bin_counts, color='#4682B4', edgecolor='black', linewidth=1.2)
plt.title('Output Phrases Length Distribution', fontsize=16, weight='bold', color='#2F4F4F')
plt.xlabel('Length Range', fontsize=14, color='#2F4F4F')
plt.ylabel('Number of Phrases', fontsize=14, color='#2F4F4F')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add data labels on bars for output
for bar in bars_output:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, int(yval), ha='center', fontsize=12, color='black')

# Show the plots
plt.tight_layout()
plt.show()

### Extra Plot

In [ ]:
# Word cloud
from wordcloud import WordCloud

input = ' '.join(input_phrases)
output = ' '.join(output_phrases)

wordcloud1 = WordCloud(width=800, height=400, background_color='white').generate(input)
wordcloud2 = WordCloud(width=800, height=400, background_color='white').generate(output)

# Plotting the bar charts
plt.figure(figsize=(20, 10))

# Plotting the input phrase lengths
plt.subplot(1, 2, 1)
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis('off')
plt.title('Question Word Cloud')

# Plotting the output phrase lengths
plt.subplot(1, 2, 2)
plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis('off')
plt.title('Answer Word Cloud')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

# Converti input_phrases in una Series
input_series = pd.Series(input_phrases)

# Applica il preprocessing
input_series = input_series.apply(preprocess)

# Conta le parole
word_counts = Counter([word for tokens in input_series for word in tokens])
most_common = word_counts.most_common(20)

# Prepara i dati per il grafico
words1, counts1 = zip(*most_common)

# Converti output_phrases in una Series
output_series = pd.Series(output_phrases)

# Applica il preprocessing
output_series = output_series.apply(preprocess)

# Conta le parole
word_counts = Counter([word for tokens in output_series for word in tokens])
most_common = word_counts.most_common(20)

# Prepara i dati per il grafico
words2, counts2 = zip(*most_common)

# Plotting the bar charts side by side
plt.figure(figsize=(20, 10))

# First subplot (Input Phrases)
plt.subplot(1, 2, 1)
sns.barplot(x=list(counts1), y=list(words1), palette='viridis')
plt.title('Top 20 Most Frequent Words in Input Phrases')
plt.xlabel('Frequency')
plt.ylabel('Words')

# Second subplot (Output Phrases)
plt.subplot(1, 2, 2)
sns.barplot(x=list(counts2), y=list(words2), palette='viridis')
plt.title('Top 20 Most Frequent Words in Output Phrases')
plt.xlabel('Frequency')
plt.ylabel('Words')

# Adjust layout and show the plots
plt.tight_layout()
plt.show()